# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постром модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверим *accuracy* на тестовой выборке.

## 1. Откроим и изучим файл

In [1]:
import sklearn
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [2]:
# прочитаем данные, присвоим наименование df
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
# изучим данные вызвав первые строки таблицы и общую информацию
df.head(5)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


### Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц сalls — количество звонков, minutes — суммарная длительность звонков в минутах, messages — количество sms-сообщений, mb_used — израсходованный интернет-трафик в Мб, is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## 2. Разобьем данные на выборки

### Разделим данные на обучающую, валидационную и тестовую в соотношении 3, 1, 1

In [5]:
# разабьем данные на выборки
# первым действием выделим обучающую часть размером 60%
df_train, df_1 = train_test_split(df, test_size=0.4, random_state=12345)
# Вторым действием разделим оставшиеся 40% выборки на валидационную и тестовую по 20% от первоначальной выборки каждая
df_valid, df_test = train_test_split(df_1, test_size=0.5, random_state=12345)

## 3. Исследуем модели

In [6]:
# разделим обучающую выборку на признаки и целевой признак
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

In [7]:
# разделим валидационную выборку на признаки и целевой признак
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

In [8]:
# разделим тестовуй выборку на признаки и целевой признак
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

In [9]:
# создадим модель для обучения решающего дерева
# для сравнения используем различную глубину дерева решений
# сравнимдолю правильных ответов в соответствии с глубиной дерева
for i in range(1, 10, 2):
    model = DecisionTreeClassifier(random_state=12345, max_depth=i)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions)
    print('max_depth =', i, accuracy)
    

max_depth = 1 0.7542768273716952
max_depth = 3 0.7853810264385692
max_depth = 5 0.7791601866251944
max_depth = 7 0.7822706065318819
max_depth = 9 0.7822706065318819


### На основании проведеного иследования видно, что наибольшая доля правильных ответов приходится на глубину дерева решений равной 3 - 78,5%

In [10]:
# создадим модель для обучения случайного леса
# для сравнения используем различное количество решающих деревьев от 10 до 50 с шагом 10 и глубиной деревьев равным 10
# сравнимдолю правильных ответов в соответствии с количеством деревьев в случайном лесу
for i in range(10, 51, 10):
    model = RandomForestClassifier(random_state=12345, n_estimators=i, max_depth=10)
    model.fit(features_train, target_train)
    accuracy = model.score(features_valid, target_valid)
    print('n_estimators =', i, accuracy)

n_estimators = 10 0.7916018662519441
n_estimators = 20 0.7916018662519441
n_estimators = 30 0.7947122861586314
n_estimators = 40 0.7962674961119751
n_estimators = 50 0.7931570762052877


### На основании проведеного иследования видно, что наибольшая доля правильных ответов приходится на количество деревьев решений равным 40 - 79,6%

In [11]:
# создадим модель для обучения логистической регрессии
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
accurasy = model.score(features_valid, target_valid)
accurasy

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7589424572317263

### На основании проведеного иследования видно, что доля правильных ответов составила - 75,8%

### В ходе иследования модели было проверины три модели: дерево решений, случайный лес и логистическая регрессия. Наибольшея доля правильных ответов была получена при применении случайного леса с количеством решающих деревьев равном 40 и глубиной деревьев равным 10. Данную модель будем ипользовать для проверки на тестовой выборке

## 4. Проверим модель на тестовой выборке

In [12]:
# используем ранее выбранную модель случацйного леса
model = RandomForestClassifier(random_state=12345, n_estimators=40, max_depth=10)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print(accuracy)

0.8009331259720062


### Проверка на тестовой выборке показала результат доли правильных ответов - 80%, что даже немного больше чем результат на валидационной выборке

## 5. (бонус) Проверим модели на адекватность

In [13]:
# Проверим модели на адекватность используя DummyClassifier 
model = DummyClassifier(random_state=12345, strategy='most_frequent')
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print(accuracy)


0.6842923794712286


### Обучив модель на основе применения простых функций видно, что доля правильных ответов по ранее расмотренным методам превосходит более чем на 10 пунктов. В целом, когда точность классификатора слишком близка к случайной, это, вероятно, означает, что что-то пошло не так: функции бесполезны, гиперпараметр настроен неправильно. В нашем случае можно считать, что расммотренные модели работают адекватно.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
